## Focusing on terse `BaseNet` training/eval

In [4]:
import os, sys
import copy as copyroot
import pandas as pd
from IPython.display import display
from matplotlib import pyplot as plt
from fastai2.basics import *
from fastai2.vision.all import *
%load_ext autoreload
%autoreload 2

from module.mnist_helpers import build_df, eda_fig_1, build_dls
from module.mnist_models import FeatsNet

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
dls = build_dls(target='topleft')
x, y = dls.one_batch()
x.shape

torch.Size([64, 1, 28, 28])

### BaseNet

In [17]:
class BaseNet(torch.nn.Module):
    def __init__(self, D_in=28, H=28, b_relu=False, b_mid=False):
        super(BaseNet, self).__init__()
        self.b_relu = b_relu
        self.b_mid  = b_mid
        
        self.flat     = nn.Flatten()
        self.linear1  = nn.Linear(in_features=D_in**2, out_features=H)
        self.relu     = nn.ReLU()
        self.linear1a = nn.Linear(in_features=H, out_features=H)
        self.linear2  = nn.Linear(in_features=H, out_features=2, bias=False)
        self.sig      = SigmoidRange(-1., 1)
    
    def forward(self, x):
        
        l0 = self.flat(x)
        l1 = self.linear1(l0)

        if self.b_relu:      l1 = self.relu(l1)
        if self.b_mid:       l1 = self.linear1a(l1)
        if self.b_mid and self.b_relu:  
                             l1 = self.relu(l1)

        l2 = self.linear2(l1)
        y =  self.sig(l2)
        return y


In [20]:
bn = BaseNet(b_relu=False, b_mid=False)
learn_bn = Learner(dls, bn, loss_func=MSELossFlat() )
learn_bn.fit(8)

epoch,train_loss,valid_loss,time
0,0.088308,0.034979,00:01
1,0.062971,0.040873,00:01
2,0.051863,0.029630,00:01
3,0.043407,0.024712,00:01
4,0.037509,0.024635,00:01
5,0.033070,0.021415,00:01
6,0.029656,0.021474,00:01
7,0.026981,0.020865,00:01


In [21]:
bn = BaseNet(b_relu=True, b_mid=False)
learn_bn = Learner(dls, bn, loss_func=MSELossFlat() )
learn_bn.fit(8)

epoch,train_loss,valid_loss,time
0,0.135725,0.048507,00:01
1,0.086116,0.040667,00:01
2,0.068733,0.034633,00:01
3,0.056620,0.027000,00:01
4,0.047974,0.025265,00:01
5,0.042075,0.024538,00:01
6,0.037453,0.022317,00:01
7,0.033733,0.022934,00:01


In [ ]:
TRIALS = 5
EPOCHS = 8
d_mse = {}

for _h in (2, 12, 28, 52):
    
    print(f'H: {_h}')
    d_mse[_h] = []
    for trial in range(TRIALS):
        
        learn = Learner(dls, BaseNet(D_in=28, H=_h), 
                        loss_func=MSELossFlat())

        with learn.no_logging(): learn.fit(EPOCHS)

        preds = learn.get_preds()
        _mse = mse(preds[0], preds[1])
        d_mse[_h].append(_mse)